### **Exploratory Analysis + Pre-Processing**

In [ ]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
import wordcloud 
from wordcloud import WordCloud
import statistics
from collections import Counter
import datetime
import textblob
from textblob import TextBlob
import sklearn

In [ ]:
import os
from google.colab import drive

# Mount google drive
DRIVE_MOUNT='/content/gdrive'
drive.mount(DRIVE_MOUNT)

# create folder to write data to
B11=os.path.join(DRIVE_MOUNT, 'My Drive', 'B11_2021')
HOMEWORK_FOLDER=os.path.join(B11, 'Project')
os.makedirs(HOMEWORK_FOLDER, exist_ok=True)

Mounted at /content/gdrive


In [ ]:
fake = pd.read_csv('/content/gdrive/MyDrive/B11_2021/Project/Fake.csv')
true = pd.read_csv('/content/gdrive/My Drive/B11_2021/Project/True.csv')
with open('/content/gdrive/My Drive/B11_2021/Project/scraped_articles.json') as json_data:
    scraped = json.load(json_data)

In [ ]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
print(list(scraped['newspapers']))

['cnn', 'bbc', 'theguardian', 'breitbart', 'infowars', 'foxnews', 'nbcnews', 'washingtonpost', 'theonion']


In [ ]:
for i, site in enumerate((list(scraped['newspapers']))):
    articles = list(scraped['newspapers'][site]['articles'])
    if i == 0:
        df = pd.DataFrame.from_dict(articles)
        df["site"] = site
    else:
        new_df = pd.DataFrame.from_dict(articles)
        new_df["site"] = site
        df = pd.concat([df, new_df], ignore_index = True)     

In [ ]:
df

,title,text,link,published,site
0,"Eyewitnesses recount bloody crackdown in Bago,...",At least 82 anti-coup protestors were killed b...,http://edition.cnn.com/videos/world/2021/04/16...,2021-04-16T00:00:00,cnn
1,This Welsh river turned white due to a milk spill,Photos and videos of the River Dulais in Wales...,http://edition.cnn.com/videos/world/2021/04/16...,2021-04-16T00:00:00,cnn
2,Hong Kong police showcase 'Chinese-style goose...,Hong Kong marked the first National Security E...,http://edition.cnn.com/videos/world/2021/04/16...,2021-04-16T00:00:00,cnn
3,"In Brazil, coronavirus killed 3 people every m...","Experts warn Brazil could soon suffer an ""unim...",http://edition.cnn.com/videos/world/2021/04/16...,2021-04-16T00:00:00,cnn
4,New sanctions imposed on Russia in response to...,The Biden administration targeted Russia with ...,http://edition.cnn.com/videos/politics/2021/04...,2021-04-15T00:00:00,cnn
...,...,...,...,...,...
1433,12 Steps to Starting a Small Business,Getty Images\n\nBeing your own boss can be imm...,https://www.nbcnews.com/veteran-services/next-...,2019-05-02T18:39:00+00:00,nbcnews
1434,Military families say this is their top concern,Members of the military face hurdles every day...,https://www.nbcnews.com/veteran-services/next-...,2019-05-28T17:17:00+00:00,nbcnews
1435,CNBC AND ACORNS ANNOUNCE STRATEGIC PARTNERSHIP,CNBC TO PROVIDE EDITORIAL AND PRODUCTION EXPER...,https://www.cnbc.com/2019/01/28/cnbc-and-acorn...,2019-01-28T00:00:00,nbcnews
1436,'Captain Tom': Funeral held for U.K. war veter...,"LONDON — ""I told you I was old,"" will be the e...",https://www.nbcnews.com/news/world/captain-tom...,2021-02-27T16:19:00+00:00,nbcnews


In [ ]:
scraped = df
scraped["label"] = scraped["site"].apply(lambda x: "fake" if(x == "breitbart" or x == "infowars" or x == "theonion") else "true")
scraped.drop(labels=["link", "site"], axis = 1, inplace=True)

In [ ]:
mask = scraped['label'] == 'fake'
scraped[mask]

,title,text,published,label
597,WATCH: BLM Activist Asks When People ‘Ready to...,An activist speaking at Black Lives Matter Pla...,2021-04-16T00:00:00,fake
598,Watch Live: Hundreds of Anti-Police Protesters...,Hundreds of anti-police and Black Lives Matter...,2021-04-16T00:00:00,fake
599,‘Jail Killer Cops’ Protester Decries U.S. for ...,"A protester in Washington, DC, blamed U.S. cap...",2021-04-16T00:00:00,fake
600,CNN’s Cuomo: If Cops Were Killing White Kids I...,CNN anchor Chris Cuomo said Friday that if pol...,2021-04-16T00:00:00,fake
601,Biden Admin to Ramp Up Experiments Using Abort...,The Biden-Harris administration’s Department o...,2021-04-16T00:00:00,fake
...,...,...,...,...
1272,Shock Footage: Man Suffers Seizure Inside Car ...,Have an important tip? Let us know. Email us h...,2021-04-16T14:18:24,fake
1273,Thousands of ‘Child Migrants’ Moved to New Eme...,Keep up to date with our latest:\n\nHave an im...,2021-04-16T14:15:00,fake
1274,Yet Another ‘Asian Hate Crime’ Victim Not Atta...,Have an important tip? Let us know. Email us h...,2021-04-16T11:42:42,fake
1275,Minneapolis BLM Icon Arrested For Failing To R...,"Chaz Neal, a prominent BLM activist in Minneso...",2021-04-16T13:26:50,fake


In [ ]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
#Create column with labels
fake["label"] = "fake"
true["label"] = "true"

In [ ]:
true.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",true
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",true
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",true
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",true
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",true


In [ ]:
fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",fake


In [ ]:
#Check shapes of dataframes. I scraped 1844 articles to add to this dataset.
print (fake.shape)
print (true.shape)
print (scraped.shape)

(23481, 5)
(21417, 5)
(1438, 4)


In [ ]:
#True news has location and publisher, which fake doesn't have. Remove for consistency
true["text"] = true['text'].apply(lambda x: x.partition("-")[2])

In [ ]:
scraped["published"] = scraped["published"].apply(lambda x: x[0:10])
scraped["published"] = scraped["published"].apply(pd.to_datetime)

In [ ]:

print (fake["subject"].unique())
print (true["subject"].unique())

['News' 'politics' 'Government News' 'left-news' 'US_News' 'Middle-east']
['politicsNews' 'worldnews']


In [ ]:
fake.drop("subject", axis=1, inplace=True)
true.drop("subject", axis=1, inplace=True)

In [ ]:
#Combine scraped with current datasets
scraped.rename(columns={"published": "date"}, inplace=True)
scraped_f = scraped[scraped["label"] == "fake"]
fake = pd.concat([fake,scraped_f], axis=0, ignore_index=True)

scraped_t = scraped[scraped["label"] == "true"]
true = pd.concat([true,scraped_t], axis=0, ignore_index=True)

In [ ]:
#Remove articles with only pictures / no text
true = true[true["text"] != ""] 
fake = fake[fake["text"] != ""] 

In [ ]:
#The datasets are pretty balanced, which is good!
print(fake.shape)
print(true.shape)

(24158, 4)
(22172, 4)


In [ ]:
fake.drop("date", axis=1, inplace=True)
true.drop("date", axis=1, inplace=True)

In [ ]:
fake.to_csv('fakeForModeling.csv')
!cp fakeForModeling.csv "/content/gdrive/My Drive/B11_2021/Project/"

In [ ]:
true.to_csv('trueForModeling.csv')
!cp trueForModeling.csv "/content/gdrive/My Drive/B11_2021/Project/"

#### Testing with different methods##

In [ ]:
fake.drop("title", axis=1, inplace=True)
true.drop("title", axis=1, inplace=True)

In [ ]:
fake.head()

,text,label
0,Donald Trump just couldn t wish all Americans ...,fake
1,House Intelligence Committee Chairman Devin Nu...,fake
2,"On Friday, it was revealed that former Milwauk...",fake
3,"On Christmas day, Donald Trump announced that ...",fake
4,Pope Francis used his annual Christmas Day mes...,fake


In [ ]:
true.head()

,text,label
0,The head of a conservative Republican faction...,true
1,Transgender people will be allowed for the fi...,true
2,The special counsel investigation of links be...,true
3,Trump campaign adviser George Papadopoulos to...,true
4,President Donald Trump called on the U.S. Pos...,true


In [ ]:
final = pd.concat([true,fake], axis=0, ignore_index=True)

In [ ]:
final.shape

(46330, 2)

In [ ]:
final.head()

,text,label
0,The head of a conservative Republican faction...,true
1,Transgender people will be allowed for the fi...,true
2,The special counsel investigation of links be...,true
3,Trump campaign adviser George Papadopoulos to...,true
4,President Donald Trump called on the U.S. Pos...,true


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
x = final['text']
y = final['label']

In [ ]:
x

0         The head of a conservative Republican faction...
1         Transgender people will be allowed for the fi...
2         The special counsel investigation of links be...
3         Trump campaign adviser George Papadopoulos to...
4         President Donald Trump called on the U.S. Pos...
                               ...                        
46325    Have an important tip? Let us know. Email us h...
46326    Keep up to date with our latest:\n\nHave an im...
46327    Have an important tip? Let us know. Email us h...
46328    Chaz Neal, a prominent BLM activist in Minneso...
46329    Have an important tip? Let us know. Email us h...
Name: text, Length: 46330, dtype: object

In [ ]:
y

0        true
1        true
2        true
3        true
4        true
         ... 
46325    fake
46326    fake
46327    fake
46328    fake
46329    fake
Name: label, Length: 46330, dtype: object

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)
y_train

45480    fake
5334     true
37234    fake
17703    true
32855    fake
         ... 
21243    true
45891    fake
42613    fake
43567    fake
2732     true
Name: label, Length: 37064, dtype: object

In [ ]:
tfvect = TfidfVectorizer(stop_words='english',max_df=0.7)
tfid_x_train = tfvect.fit_transform(x_train)
tfid_x_test = tfvect.transform(x_test)

In [ ]:
classifier = PassiveAggressiveClassifier(max_iter=50)
classifier.fit(tfid_x_train,y_train)

PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=50, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [ ]:
y_pred = classifier.predict(tfid_x_test)
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 98.29%


In [ ]:
cf = confusion_matrix(y_test,y_pred, labels=['fake','true'])
print(cf)

[[4791   73]
 [  85 4317]]


In [ ]:
def fake_news_det(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = classifier.predict(vectorized_input_data)
    print(prediction)

In [ ]:
fake_news_det('U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.')

['true']


In [ ]:
fake_news_det("""Go to Article 
President Barack Obama has been campaigning hard for the woman who is supposedly going to extend his legacy four more years. The only problem with stumping for Hillary Clinton, however, is sheâ€™s not exactly a candidate easy to get too enthused about.  """)

['fake']


Binomial Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# 2. instantiate a logistic regression model
lr = LogisticRegression()

lr.fit(tfid_x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred_lr = lr.predict(tfid_x_test)
score = accuracy_score(y_test,y_pred_lr)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 96.84%


In [ ]:
cf = confusion_matrix(y_test,y_pred_lr, labels=['fake','true'])
print(cf)

[[4718  146]
 [ 147 4255]]


Support Vector Machines

In [ ]:
from sklearn.svm import SVC

#Linear kernel fits decently well, decided not to use another kernel because of parsimony 
#and because linear has the lowest risk of overfitting
svc = SVC(kernel='linear', random_state=1) 
svc.fit(tfid_x_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=1, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
y_pred_svm = svc.predict(tfid_x_test)
score = accuracy_score(y_test,y_pred_svm)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 97.8%


In [ ]:

cf = confusion_matrix(y_test,y_pred_lr, labels=['fake','true'])
print(cf)

[[4718  146]
 [ 147 4255]]
